# Окончательный вариант парсинга

# подготовка к парсингу

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

импорт библиотек

In [2]:
import warnings
warnings.filterwarnings('ignore')

отключение ворнингов

In [3]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

задание параметров таблиц

In [4]:
def find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

функция создания столбца с кодом товара

In [5]:
def find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n',''))
    return xxx  

функция создания столбца с парсингом наличия товара

In [6]:
def find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

функция создающая столбец с названием товара

In [7]:
def find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

функция создающая столбец с продажной ценой товара

In [8]:
def find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

функция созжающая столбец с предидущей ценой для акционного товара

In [9]:
def find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

Функция создающая столбец с урл адресом товара

In [10]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

Функция для проверки датафрейма(вид дублекаты пропуски)

In [11]:
def parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        while q>0:        
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
            else:
                q = 0
    data['id'] = data['url'].apply(find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(find_availability)
    data['title'] = data['url'].apply(find_title)
    data['price'] = data['url'].apply(find_price)
    data['old_price'] = data['url'].apply(find_old_price)
    data['url'] = data['url'].apply(find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

Функция предварительного парсинга сайта, которая проходит по всем страницам в урл адресах переменной `all_url` и создает датафрейм в котором храняться: урл адреса товаров, индентефикатор товаров, дата предварительного парсинга товара, наличия товара, названия товара, цены товара, старой цены для акционного товара.

# Парсинг цен кодов товара и url

In [12]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

all_url = ['https://hi-tech.md/televizory-i-elektronika/page-']

all_url = ['https://hi-tech.md/bytovaya-tehnika/page-']

all_url = ['https://hi-tech.md/kompyuternaya-tehnika/page-']

all_url = ['https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-']

all_url = ['https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-']

all_url = ['https://hi-tech.md/tovary-dlya-doma/page-']

переменная со списком ссылок на сайт для парсинга

In [ ]:
df = parsing_2(all_url)
df

  0%|          | 0/6 [00:00<?, ?it/s]

предварительный парсинг

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

удаление дублекатов

In [ ]:
df.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

In [ ]:
df = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

загрузка резервной копии парсинга данных

df = df.sample(30)

тестовое уменьшение таблици

In [ ]:
def xxx(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    element = soup.findAll('div', class_='ty-warehouses-store__availability')
    elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
    name = soup.findAll('span', class_='ty-breadcrumbs__current')
    try:
        d_0 = (element[0].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_0 = (np.NaN)
    try:
        d_1 = (element[1].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_1 = (np.NaN)
    try:
        d_2 = (element[2].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_2 = (np.NaN)
    try:
        d_3 = (element[3].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_3 = (np.NaN)
    try:
        d_4 = (element[4].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_4 = (np.NaN)
    try:
        d_5 = (element[5].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_5 = (np.NaN)
    try:
        d_6 = (element[6].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_6 = (np.NaN)
    try:
        d_7 = (element[7].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_7 = (np.NaN)
    try:
        d_8 = (element[8].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_8 = (np.NaN)
    try:
        d_9 = (element[9].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_9 = (np.NaN)
    try:
        d_10 = (element[10].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_10 = (np.NaN)
    try:
        d_11 = (element[11].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_11 = (np.NaN)
    try:
        d_12 = (element[12].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_12 = (np.NaN)
    try:
        d_13 = (element[13].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    except:
        d_13 = (np.NaN)
        
    try:
        names = name[0].text
        name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
        err = 'err'
    except:
        names = 'Noname'
        name_group = 'Noname/Noname/Noname/Noname/Noname/Noname'
        err = 'Noname_err'
    try:
        len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))
    except:
        len_name = 1
        
    try:
        if len_name <= 1:
            d_14 = 'err1'
            d_15 = 'err1'
            d_16 = 'err1'
            d_17 = 'err1'
        elif len_name <= 2:
            d_14 = (name_group.split("/")[1])
            d_15 = (name_group.split("/")[1])
            d_16 = (name_group.split("/")[1])
            d_17 = (name_group.split("/")[1])        
        elif len_name <= 3:
            d_14 = (name_group.split("/")[1])
            d_15 = (name_group.split("/")[2])
            d_16 = (name_group.split("/")[2])
            d_17 = (name_group.split("/")[2])
        elif len_name == 4:
            d_14 = (name_group.split("/")[1])
            d_15 = (name_group.split("/")[2])
            d_16 = (name_group.split("/")[3])  
            d_17 = (name_group.split("/")[3])
        elif len_name > 4:
            d_14 = (name_group.split("/")[1])
            d_15 = (name_group.split("/")[2])
            d_16 = (name_group.split("/")[3])  
            d_17 = (name_group.split("/")[4])
    except:
        d_14 = 'err0'
        d_15 = 'err0'
        d_16 = 'err0'
        d_17 = 'err0'
        
    return pd.Series([(d_0), (d_1), (d_2), (d_3), (d_4), (d_5), (d_6), (d_7), (d_8), (d_9), (d_10), (d_11), (d_12), (d_13), (d_14), (d_15), (d_16), (d_17)])

Функция основного парсинга, которая для каждой строки по урл адресу парсит и находит наличие товара в магахинах а также группу товара, категорию товара, подкатегорию товата и количество подкатегорий товара

# Парсинг остатков и групп товара

In [ ]:
tqdm.pandas() 
df[['тир_юности_14а','тир_25_октября_69','бенд_спортивная_5','бенд_советская_5','бенд_советская_8','слободзея','днестровск',
   'первомайск','григ_кмаркса_181а','григ_кмаркса_172','дубоссары','рыбн_вальченко_43','рыбн_кирова_135','каменка',
   '_category','_subcategory','_group','_subgroup']] = df['url'].progress_apply(xxx)

reqx = requests.get('https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-transparent-led15w-e27-5628-tkl/')
soup = bs(reqx.text, 'html.parser')    
element = soup.findAll('div', class_='ty-warehouses-store__availability')
elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
name = soup.findAll('span', class_='ty-breadcrumbs__current')

names = name[0].text
name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
err = 'err'


len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))


if len_name <= 1:
    d_14 = 'err1'
    d_15 = 'err1'
    d_16 = 'err1'
    d_17 = 'err1'
elif len_name <= 2:
    d_14 = (name_group.split("/")[1])
    d_15 = (name_group.split("/")[1])
    d_16 = (name_group.split("/")[1])
    d_17 = (name_group.split("/")[1])        
elif len_name <= 3:
    d_14 = (name_group.split("/")[1])
    d_15 = (name_group.split("/")[2])
    d_16 = (name_group.split("/")[2])
    d_17 = (name_group.split("/")[2])
elif len_name == 4:
    d_14 = (name_group.split("/")[1])
    d_15 = (name_group.split("/")[2])
    d_16 = (name_group.split("/")[3])  
    d_17 = (name_group.split("/")[3])
elif len_name > 4:
    d_14 = (name_group.split("/")[1])
    d_15 = (name_group.split("/")[2])
    d_16 = (name_group.split("/")[3])  
    d_17 = (name_group.split("/")[4])
print(d_14, d_15, d_16, d_17)

In [ ]:
df._category.unique()

In [ ]:
df._subcategory.unique()

In [ ]:
df._group.unique()

In [ ]:
df._subgroup.unique()

Основной парсинг

In [ ]:
df[df['_group']=='err_3']

In [ ]:
preprocessing(df)

In [ ]:
df.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

In [ ]:
df = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

загрузка резервной копии парсинга данных

In [ ]:
df._category.unique()

In [ ]:
df['error_count'] = 0
df.loc[df['каменка'].isna(), 'error_count'] = 1
df['error_group'] = 0
df.loc[df['_subgroup'].isna(), 'error_group'] = 1


Создание столбцов с флагом ошибки для товаров, для которых нет данных по остаткам хотябы в одном магазине

In [ ]:
df.loc[df['error_count']==1 , ['тир_юности_14а', 'тир_25_октября_69', 'бенд_спортивная_5',
       'бенд_советская_5', 'бенд_советская_8', 'слободзея', 'днестровск',
       'первомайск', 'григ_кмаркса_181а', 'григ_кмаркса_172', 'дубоссары',
       'рыбн_вальченко_43', 'рыбн_кирова_135', 'каменка']] = 0
df.loc[df['error_group']==1 , ['_category',
       '_subcategory', '_group', '_subgroup']] = 'No_group'

удаление наличия товара в магазинах и наименовании группы если у товара есть ошибки в этих данны

In [ ]:
df['тир_юности_14а'] = df['тир_юности_14а'].astype('float')
df['тир_25_октября_69'] = df['тир_25_октября_69'].astype('float')
df['тирасполь'] = df['тир_юности_14а'] + df['тир_25_октября_69']
df['бенд_спортивная_5'] = df['бенд_спортивная_5'].astype('float')
df['бенд_советская_5'] = df['бенд_советская_5'].astype('float')
df['бенд_советская_8'] = df['бенд_советская_8'].astype('float')
df['бендеры'] = df['бенд_спортивная_5'] + df['бенд_советская_5'] + df['бенд_советская_8']
df['слободзея'] = df['слободзея'].astype('float')
df['днестровск'] = df['днестровск'].astype('float')
df['первомайск'] = df['первомайск'].astype('float')
df['григ_кмаркса_181а'] = df['григ_кмаркса_181а'].astype('float')
df['григ_кмаркса_172'] = df['григ_кмаркса_172'].astype('float')
df['григориополь'] = df['григ_кмаркса_181а'] + df['григ_кмаркса_172']
df['дубоссары'] = df['дубоссары'].astype('float')
df['рыбн_вальченко_43'] = df['рыбн_вальченко_43'].astype('float')
df['рыбн_кирова_135'] = df['рыбн_кирова_135'].astype('float')
df['рыбница'] = df['рыбн_вальченко_43'] + df['рыбн_кирова_135']
df['каменка'] = df['каменка'].astype('float')
df['all_shop'] = (df['тирасполь'] + df['бендеры'] + df['слободзея'] + df['днестровск'] +
                  df['первомайск'] + df['григориополь'] + df['дубоссары'] + df['рыбница'] + df['каменка'])                 

Изменение типов данных столбцов наличия товара в определенном магазине на тип данных `float`  и создания столбцов по наличию товара в магазинах городов

In [ ]:
df = df[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'title',
               'date',   'price', 'old_price', 'availability', 'all_shop', 'тирасполь',
               'тир_юности_14а', 'тир_25_октября_69', 'бендеры', 'бенд_спортивная_5',
               'бенд_советская_5', 'бенд_советская_8', 'рыбница', 'рыбн_вальченко_43', 
               'рыбн_кирова_135', 'дубоссары', 'днестровск', 'слободзея', 'первомайск',
               'каменка', 'григориополь','григ_кмаркса_181а', 'григ_кмаркса_172',
               'error_count', 'error_group', ]]

изменение порядка располажения столбцов

In [ ]:
preprocessing(df)

просмотр данных после парсинга

# Группировка товара

- разобраться с креслами в ногруп

In [ ]:
df.loc[df['_subcategory']=='Мультимедиа', '_subcategory'] = 'Аудиосистемы'
df.loc[df['_group']=='Акустические системы', '_group'] = 'Акустика'
df.loc[df['_group']=='Музыкальная техника', '_group'] = 'Акустика'
df.loc[df['_subcategory']=='Умный дом с Яндекс','_subcategory'] = 'Аудиосистемы'
df.loc[df['_group']=='Яндекс Станции', '_group'] = 'Акустика'
df.loc[df['_group']=='AirPods', '_subcategory'] = 'Аудиосистемы'
df.loc[df['_group']=='AirPods', '_group'] = 'Наушники'
df.loc[df['_group']=='Apple Watch', '_group'] = 'Часы и браслеты'
df.loc[df['_group']=='iPhone', '_group'] = 'Смартфоны'
df.loc[df['_group']=='iPad', '_group'] = 'Планшеты'
df.loc[df['_group']=='MacBook', '_group'] = 'Ноутбуки'
df.loc[df['_group']=='iMac', '_group'] = 'Моноблоки'
df.loc[df['_group']=='Часы и браслеты', '_subcategory'] = 'Гаджеты'
df.loc[df['_group']=='Смартфоны', '_subcategory'] = 'Смартфоны и аксессуары'
df.loc[df['_group']=='Планшеты', '_subcategory'] = 'Готовые решения'
df.loc[df['_group']=='Ноутбуки', '_subcategory'] = 'Готовые решения'
df.loc[df['_group']=='Моноблоки', '_subcategory'] = 'Готовые решения'
df.loc[df['_subcategory']=='Стиральные и сушильные машины', '_group'] = 'Стиральные машины'
df.loc[df['_subgroup']=='Сушильные машины', '_group'] = 'Сушильные машины'
df.loc[df['_subgroup']=='Наклейки', '_group'] = 'Наклейки'
df.loc[df['_subgroup']=='Подставки', '_group'] = 'Подставки'
df.loc[df['_subgroup']=='Замки', '_group'] = 'Замки'
df.loc[df['_subgroup']=='USB HUB', '_group'] = 'USB HUB'
df.loc[df['_subgroup']=='Сумки и чехлы', '_group'] = 'Сумки и чехлы'
df.loc[df['_subgroup']=='Защитные стекла', '_group'] = 'Защитные стекла'
df.loc[df['_subgroup']=='Сумки и чехлы', '_group'] = 'Сумки и чехлы'
df.loc[df['_subcategory']=='Мониторы','_group'] = 'Мониторы'

In [ ]:
for i in df['_subcategory'].unique():
    t = df[df['_subcategory']==i]
    print(i)
    print(t['_group'].unique())
    print('-'*100)

просмотр групп товаров в подкатегориях

In [ ]:
for i in df['_group'].unique():
    t = df[df['_group']==i]
    print(i)
    print(t['_subgroup'].unique())
    print('-'*100)

In [ ]:
df._category.unique()

In [ ]:
df._subcategory.unique()

In [ ]:
df._group.unique()

In [ ]:
df._subgroup.unique()

# Подготовка анализа

In [ ]:
df.head(2)

In [ ]:
brend = pd.read_excel('csv/brend.xlsx')

In [ ]:
def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

In [ ]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

создаем столбец с брендами

In [ ]:
df['action_sale'] = 0
df.loc[df['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [ ]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [ ]:
def count_group(x):
    return df[df['_group']==x]['id'].count()    
tqdm.pandas()
df['count_group'] = df['_group'].progress_apply(count_group)

создание столбца с количеством товара в группе

In [ ]:
def count_group_presence(x):
    return df[(df['availability']=='В наличии')&(df['_group']==x)]['id'].count()   
tqdm.pandas()
df['activ_count_group'] = df['_group'].progress_apply(count_group_presence)

создание столбца с количеством товара, который есть в наличии, в группе

In [ ]:
def dif_price(x):    
    max_price = df[(df['availability']=='В наличии')&(df['_group']==x)]['price'].max()
    min_price = df[(df['availability']=='В наличии')&(df['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
df['dif_price'] = df['_group'].progress_apply(dif_price)

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [ ]:
df['group_prise'] = df.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [ ]:
df = df[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'all_shop', 'count_group', 'activ_count_group',
         'тирасполь', 'тир_юности_14а', 'тир_25_октября_69', 'бендеры',
         'бенд_спортивная_5', 'бенд_советская_5', 'бенд_советская_8', 'рыбница',
         'рыбн_вальченко_43', 'рыбн_кирова_135', 'дубоссары', 'днестровск',
         'слободзея', 'первомайск', 'каменка', 'григориополь',
         'григ_кмаркса_181а', 'григ_кмаркса_172', 'error_count', 'error_group']]

In [ ]:
df.to_csv('csv/hi_tech_end_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False) 

# Аналитика акционных товаров

In [ ]:
df

In [ ]:
df.head(2)

In [ ]:
df.groupby('_group')['price'].mean().reset_index()

t = df.groupby('_group').agg({'price':['count','mean','median']}).reset_index()
t.columns = ['_'.join(col).strip() for col in t.columns.values]
t

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
df[df['_group']=='Пылесосы'].head(2)

In [ ]:
separator = ['_group','brend','group_prise']

In [ ]:
for x in separator:
    for i in df['_subcategory'].unique():        
        plt.figure(figsize=(16,9))
        sns.barplot(df[df['_subcategory']==i].groupby([x ,'action_sale'])['id'].count().sort_values(ascending=False).reset_index(),
                       x=x, y='id', hue='action_sale')
        plt.xticks(rotation=45)
        plt.title(f'количество товаров разных {x} в подкатегории {i} распределенные по наличию акции ')
        plt.xlabel(f' {i}')
        plt.ylabel('количество товара')
        plt.show()

for x in separator:
    for i in df['_subcategory'].unique():        
        plt.figure(figsize=(16,9))
        sns.barplot(df[df['_subcategory']==i].groupby([x ,'action_sale'])['price'].mean().reset_index()
                    .sort_values('price',ascending=False),x=x, y='price', color='blue', errorbar=None)
        plt.xticks(rotation=45)
        plt.title(f'средняя цена для разных{x} в подкатегории {i}')
        plt.xlabel(f'{i}')
        plt.ylabel('средняя цена')
        plt.show()